In [66]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

,date,F1,F2,F1 Running Total,F2 Running Total
0,2020-12-1,10,5,10,5
1,2020-12-3,9,6,19,11
2,2020-12-4,7,16,26,27


In [188]:
d="""Date/time,Flag,Type,cuml_sell_qty,cuml_buy_qty,expected col
2021-03-05 13:00:00,1,BOD,0.000,0.000 
2021-03-05 13:00:52,2,action,-0.008,0.000,-0.008
2021-03-05 13:00:53,2,action,-0.545,0.000,-0.545
2021-03-06 01:00:55,2,action,-0.545,0.019,-0.526
2021-03-06 13:00:00,3,BOD,,,-0.526
2021-03-06 13:00:54,4,action,,0.176,-0.350
2021-03-07 01:01:08,4,action,,0.204,-0.322
2021-03-07 13:00:00,5,BOD,,0.204,-0.147
2021-03-08 01:06:04,6,action,,0.013,-0.309
2021-03-08 13:00:00,7,BOD,,,-0.309
2021-03-08 13:01:02,8,action,,0.309,0.000
2021-03-09 01:00:55,8,action,-0.296,0.309,-0.296"""
df=g(d)
df

,Date/time,Flag,Type,cuml_sell_qty,cuml_buy_qty,expected col
0,2021-03-05 13:00:00,1,BOD,0.000,0.000,NaN
1,2021-03-05 13:00:52,2,action,-0.008,0.000,-0.008
2,2021-03-05 13:00:53,2,action,-0.545,0.000,-0.545
3,2021-03-06 01:00:55,2,action,-0.545,0.019,-0.526
4,2021-03-06 13:00:00,3,BOD,NaN,NaN,-0.526
5,2021-03-06 13:00:54,4,action,NaN,0.176,-0.350
6,2021-03-07 01:01:08,4,action,NaN,0.204,-0.322
7,2021-03-07 13:00:00,5,BOD,NaN,0.204,-0.147
8,2021-03-08 01:06:04,6,action,NaN,0.013,-0.309
9,2021-03-08 13:00:00,7,BOD,NaN,NaN,-0.309


In [215]:
weekmask_egypt = "Sun Mon Tue Wed Thu"
# They also observe International Workers' Day so let's # add that for a couple of years
holidays=["2018-05-01",datetime.datetime(2019,5,1),
          np.datetime64("2020-05-01"),]
bday_egypt = pd.offsets.CustomBusinessDay(
    holidays = holidays,
    weekmask = weekmask_egypt
    )
dt = datetime.datetime(2021,4,30)
dt + 2 * bday_egypt # Timestamp('2021-05-03 00:00:00')

bday_egypt.apply(dt) # Timestamp('2021-05-02 00:00:00')

Timestamp('2021-05-02 00:00:00')

In [226]:
dts = pd.date_range(dt, periods=5, freq=bday_egypt)
pd.Series(dts.weekday, dts).map(pd.Series("Mon Tue Wed Thu Fri Sat Sun".split()))

2021-05-02    Sun
2021-05-03    Mon
2021-05-04    Tue
2021-05-05    Wed
2021-05-06    Thu
Freq: C, dtype: object

In [228]:
import pandas.tseries.holiday
print(pandas.tseries.holiday.USFederalHolidayCalendar)

<class 'pandas.tseries.holiday.USFederalHolidayCalendar'>


In [258]:
df=pd.DataFrame({'month':['1','1','1','1','1','2','2','2','2','2','2','2'],'X1': 
[30,42,25,32,12,10,4,6,5,10,24,21],'X2':[10,76,100,23,65,94,67,24,67,54,87,81],'X3': 
[23,78,95,52,60,76,68,92,34,76,34,12]})  
df

,month,X1,X2,X3
0,1,30,10,23
1,1,42,76,78
2,1,25,100,95
3,1,32,23,52
4,1,12,65,60
5,2,10,94,76
6,2,4,67,68
7,2,6,24,92
8,2,5,67,34
9,2,10,54,76


In [262]:
from sklearn.cluster import KMeans
cols = df.columns[2:4]
mapping = {0: 'weak', 1: 'average', 2: 'best'}


def cluster(X):
    k_means = KMeans(n_clusters=3).fit(X)
    return X.groupby(k_means.labels_)\
        .transform('mean').sum(1)\
        .rank(method='dense').sub(1)\
        .astype(int).to_frame()

df['Cluster_id'] = df.groupby('month')[cols].apply(cluster)
df['Cluster_cat'] = df['Cluster_id'].map(mapping)

ModuleNotFoundError: No module named 'sklearn'

In [7]:
d="""a,b
20,30
40,50"""
df=pd.read_csv(StringIO(d))
df

,a,b
0,20,30
1,40,50


In [22]:
d="""DEAL ID,DATE
432,20200405
522,20180405
632,20180409
9332,20190405"""
df=g(d)
df

,DEAL ID,DATE
0,432,20200405
1,522,20180405
2,632,20180409
3,9332,20190405


In [28]:
new_df = df.groupby(pd.to_datetime(df.DATE, format='%Y%m%d').dt.year).agg({'DEAL ID' : 'count'})
new_df

,DEAL ID
DATE,
2018,2
2019,1
2020,1


In [3]:
d="""C,A,V,D
9,apar,1.0,0
8,bpar,4.0,8
7,cpar,7.0,7
0,apar,8.0,6
8,apar,9.0,4
9,bpar,3.0,2"""
df=g(d)
df

,C,A,V,D
0,9,apar,1.0,0
1,8,bpar,4.0,8
2,7,cpar,7.0,7
3,0,apar,8.0,6
4,8,apar,9.0,4
5,9,bpar,3.0,2


In [13]:
pd.concat(df, np.where(df.A.eq('apar'), [df.A+'_t',df.V-.5], np.nan)

array([['apar_t', nan, nan, 'apar_t', 'apar_t', nan],
       [0.5, nan, nan, 7.5, 8.5, nan]], dtype=object)

In [15]:
pd.DataFrame(np.where(df.A.eq('apar'), [df.A+'_t',df.V-.5], np.nan)).T

,0,1
0,apar_t,0.5
1,NaN,NaN
2,NaN,NaN
3,apar_t,7.5
4,apar_t,8.5
5,NaN,NaN


In [25]:
df=pd.DataFrame({
    'Month':['2020-11', '2020-11', '2020-11', '2020-12', '2020-12', '2020-12',
            '2021-01','2021-01','2021-02','2021-02','2021-02','2021-02'],
    'ID':['A','B','B','A','A','C','A','A','C','A','A','B'],
    'f_count':[4,2,3,1,5,3,2,1,1,1,2,3],
    't_count':[5,4,1,0,8,4,1,0,2,3,5,7,]
})

df['Month'] = pd.to_datetime(df.Month)
df = df.set_index('Month')

df1 = df.last('M').groupby('ID').sum().reset_index().rename(
    columns={'f_count':'f_count(current month)',
            't_count':'t_count(current month)'})
df2 = df.last('3M').first('2M').groupby('ID').sum().reset_index().rename(
    columns={'f_count':'f_count(past 3 months)',
            't_count':'t_count(past 3 months)'})

df  = pd.merge(df1, df2, on='ID', how='inner').reindex(columns = [ 'ID',
    'f_count(current month)', 'f_count(past 3 months)',
    't_count(current month)','t_count(past 3 months)'
])
df

,ID,f_count(current month),f_count(past 3 months),t_count(current month),t_count(past 3 months)
0,A,3,9,8,9
1,C,1,3,2,4


In [34]:
d="""Timestamp,Event,SID,User
2021-04-15 10:35,OPEN,7933711,user2
2021-04-15 10:37,OPEN,8022822,user1
2021-04-15 10:37,OPEN,7933711,user2
2021-04-15 10:41,OPEN,1234567,user2
2021-04-15 10:45,OPEN,1234567,user2
2021-04-15 11:39,OPEN,8022822,user1
2021-04-15 11:59,DO,7933711,user2
2021-04-15 12:24,OPEN,7933711,user2
2021-04-15 12:59,DO,8022822,user1
2021-04-15 13:25,OPEN,8022822,user1
2021-04-15 13:29,OPEN,7933711,user2
2021-04-15 14:27,OPEN,8022822,user1"""
df=g(d)
df.tail()

,Timestamp,Event,SID,User
7,2021-04-15 12:24,OPEN,7933711,user2
8,2021-04-15 12:59,DO,8022822,user1
9,2021-04-15 13:25,OPEN,8022822,user1
10,2021-04-15 13:29,OPEN,7933711,user2
11,2021-04-15 14:27,OPEN,8022822,user1


In [35]:
df['Timestamp'] = pd.to_datetime(df.Timestamp)
#df = df.set_index('Timestamp')
df

,Timestamp,Event,SID,User
0,2021-04-15 10:35:00,OPEN,7933711,user2
1,2021-04-15 10:37:00,OPEN,8022822,user1
2,2021-04-15 10:37:00,OPEN,7933711,user2
3,2021-04-15 10:41:00,OPEN,1234567,user2
4,2021-04-15 10:45:00,OPEN,1234567,user2
5,2021-04-15 11:39:00,OPEN,8022822,user1
6,2021-04-15 11:59:00,DO,7933711,user2
7,2021-04-15 12:24:00,OPEN,7933711,user2
8,2021-04-15 12:59:00,DO,8022822,user1
9,2021-04-15 13:25:00,OPEN,8022822,user1


In [48]:
df.Timestamp = pd.to_datetime(df.Timestamp) # if not already datetime

df = df.sort_values(['User', 'Timestamp', 'SID', 'Event'])
print(df)
df.Event.eq('DO').shift()

             Timestamp Event      SID   User
1  2021-04-15 10:37:00  OPEN  8022822  user1
5  2021-04-15 11:39:00  OPEN  8022822  user1
8  2021-04-15 12:59:00    DO  8022822  user1
9  2021-04-15 13:25:00  OPEN  8022822  user1
11 2021-04-15 14:27:00  OPEN  8022822  user1
0  2021-04-15 10:35:00  OPEN  7933711  user2
2  2021-04-15 10:37:00  OPEN  7933711  user2
3  2021-04-15 10:41:00  OPEN  1234567  user2
4  2021-04-15 10:45:00  OPEN  1234567  user2
6  2021-04-15 11:59:00    DO  7933711  user2
7  2021-04-15 12:24:00  OPEN  7933711  user2
10 2021-04-15 13:29:00  OPEN  7933711  user2


1       NaN
5     False
8     False
9      True
11    False
0     False
2     False
3     False
4     False
6     False
7      True
10    False
Name: Event, dtype: object

In [49]:
df = pd.DataFrame({ 
    'BU': ['Total', 'Total', 'Total', 'CRS', 'CRS', 'CRS'], 
    'Line_Item': ['Revenues','EBT', 'Expenses', 'Revenues', 'EBT', 'Expenses'],
    '1Q16-1': [100, 120, 0, 200, 190, 210],
    '2Q16-1': [100, 0, 130, 200, 190, 210],
    '3Q16-1': [200, 250, 0, 120, 0, 190],
    '1Q16-2': [100, 120, 0, 200, 190, 210],
    '2Q16-2': [100, 0, 130, 200, 190, 210],
    '3Q16-2': [200, 250, 0, 120, 0, 190]
})
df

,BU,Line_Item,1Q16-1,2Q16-1,3Q16-1,1Q16-2,2Q16-2,3Q16-2
0,Total,Revenues,100,100,200,100,100,200
1,Total,EBT,120,0,250,120,0,250
2,Total,Expenses,0,130,0,0,130,0
3,CRS,Revenues,200,200,120,200,200,120
4,CRS,EBT,190,190,0,190,190,0
5,CRS,Expenses,210,210,190,210,210,190


In [55]:
df.filter(like=*[f'{for i in range(1,4)}Q16'])

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-55-052602e4f9b3>, line 1)

In [65]:
df.assign(**{
    f'{i}Q16': df.filter(like=f'{i}Q16').sum(1) for i in [1,2,3]
})


print(df.filter(like=f'{i}Q16').sum(1) for i in [1,2,3])


<generator object <genexpr> at 0x12d75bb30>


In [69]:
d3 = pd.DataFrame({
    'DIST_EVENT':[1,2,0,3],
    'DIST_NON_EVENT':[2,3,4,0]
})

In [72]:
np.where(d3.DIST_EVENT * d3.DIST_NON_EVENT == 0 ,np.nan ,np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT))

/Users/utsav/.conda/envs/myenv/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


array([-0.69314718, -0.40546511,         nan,         nan])

In [86]:
df= pd.DataFrame({
    'Country':['a','b','a','b'],
    'Height':[10,20,30,40]
})

df.groupby("Country", as_index=False).Height.mean().sort_values('Height', ascending=False)

,Country,Height
1,b,30
0,a,20


In [111]:
d="""yearmon,college,major,gpa,num
20140401,1,a,3.36,29
20180401,2,b,2.63,48
20160401,3,c,3.23,55
20170401,4,d,4.22,1
20140401,3,b,3.72,72"""
df=g(d)
df

,yearmon,college,major,gpa,num
0,20140401,1,a,3.36,29
1,20180401,2,b,2.63,48
2,20160401,3,c,3.23,55
3,20170401,4,d,4.22,1
4,20140401,3,b,3.72,72


In [112]:
df = df.groupby('major').apply(lambda x: np.average(x['gpa'], weights=x['num']))
df

major
a    3.360
b    3.284
c    3.230
d    4.220
dtype: float64

In [142]:
import pandas as pd

data = {'Company': ["0652369- INTER SUPPORT LLP, 202011","CIRCLE TRADING LTD 1-593616, 2020-06, 0201","Area  Food Service Co., Ltd.-6958047, 2020-07"]}

df = pd.DataFrame(data)

df['co'] = df['Company'].str.extract('(\d{5,})')
print (df['co'])

0    0652369
1     593616
2    6958047
Name: co, dtype: object
